In [ ]:
import sqlite3
from pathlib import Path
import pandas as pd

In [28]:
# Creating the file
Path('../Data/ILINet.sqlite').touch()

In [29]:
# Connecting the file
conn = sqlite3.connect('../Data/ILINet.sqlite')
c = conn.cursor()

In [76]:
# c.execute('''DROP TABLE ilinet''')

In [77]:
# Creating the table
c.execute('''CREATE TABLE ilinet (REGIONTYPE text,REGION text,YEAR int,WEEK int,
PERCENTWEIGHTEDILI float,PERCENTUNWEIGHTEDILI float,
AGE04 int,AGE2549 int,AGE2564 int,AGE524 int,AGE5064 int,AGE65 int,
ILITOTAL int,NUMOFPROVIDERS int,TOTALPATIENTS int,ID int NOT NULL PRIMARY KEY)''')

Working with the data frame to include a primary key manually

In [78]:
# load the data into a Pandas DataFrame
ili_df = pd.read_csv('../Data/ILINet.csv')
ili_df.head()

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514


In [80]:
ili_df["ID"] = 0

In [81]:
i = 0
for i in range(len(ili_df)):
    ili_df["ID"][i] = i
    i += 1

C:\Users\micha\AppData\Local\Temp\ipykernel_24932\3753259612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ili_df["ID"][i] = i


In [82]:
ili_df.head()

,REGIONTYPE,REGION,YEAR,WEEK,PERCENTWEIGHTEDILI,PERCENTUNWEIGHTEDILI,AGE04,AGE2549,AGE2564,AGE524,AGE5064,AGE65,ILITOTAL,NUMOFPROVIDERS,TOTALPATIENTS,ID
0,States,Alabama,2013,40,X,1.59044,X,X,X,X,X,X,201,27,12638,0
1,States,Alaska,2013,40,X,0.6238,X,X,X,X,X,X,13,9,2084,1
2,States,Arizona,2013,40,X,0.792311,X,X,X,X,X,X,230,59,29029,2
3,States,Arkansas,2013,40,X,1.63326,X,X,X,X,X,X,66,18,4041,3
4,States,California,2013,40,X,1.52479,X,X,X,X,X,X,633,137,41514,4


In [83]:
# write the data to a sqlite table
ili_df.to_sql('ilinet', conn, if_exists='append', index = False)

8478

In [85]:
# Checking that it loaded correctly
c.execute('''SELECT * FROM ilinet''').fetchone()

[('States',
  'Alabama',
  2013,
  40,
  'X',
  1.59044,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  201,
  27,
  12638,
  0),
 ('States',
  'Alaska',
  2013,
  40,
  'X',
  0.6238,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  13,
  9,
  2084,
  1),
 ('States',
  'Arizona',
  2013,
  40,
  'X',
  0.792311,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  230,
  59,
  29029,
  2),
 ('States',
  'Arkansas',
  2013,
  40,
  'X',
  1.63326,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  66,
  18,
  4041,
  3),
 ('States',
  'California',
  2013,
  40,
  'X',
  1.52479,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  633,
  137,
  41514,
  4),
 ('States',
  'Colorado',
  2013,
  40,
  'X',
  0.652244,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  142,
  9,
  21771,
  5),
 ('States',
  'Connecticut',
  2013,
  40,
  'X',
  0.701388,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  47,
  27,
  6701,
  6),
 ('States',
  'Delaware',
  2013,
  40,
  'X',
  0.49636,
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  15,
  

In [86]:
# Removing most of the null data
c.execute('''ALTER TABLE ilinet DROP COLUMN REGIONTYPE''')
c.execute('''ALTER TABLE ilinet DROP COLUMN PERCENTWEIGHTEDILI''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE04''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2549''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE2564''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE524''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE5064''')
c.execute('''ALTER TABLE ilinet DROP COLUMN AGE65''')



In [88]:
# Checking for a final time
c.execute('''SELECT * FROM ilinet''').fetchone()

('Alabama', 2013, 40, 1.59044, 201, 27, 12638, 0)

Parse the db so we can load the api from it

In [89]:
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [90]:
Base = automap_base()

In [91]:
# Setting up the DB
engine = create_engine("sqlite:///../Data/ILINet.sqlite")

In [92]:
# Reflect the tables
Base.prepare(autoload_with=engine)

In [93]:
# Save the table reference
Base.classes.keys()

['ilinet']